# Story

In this example, we will simulate one process spreading through a multilayer network with two layers. That kind of experiments are possible to perform via the `Network Diffusion`, however they require some additional 'hacky' work to do.

Colleges communicate via social network and during face-to-face meetings. All of them know each other, but people usually work remotely during the pandemic, and our guys are no exception from the rule. Hence, some don't go to the office - we will simulate it by disconnecting nodes in the 'work' layer. Moreover, it's obvious that the Internet has a more significant impact on information spreading than real interactions. By that reason, in our model, transitions for a social network will be higher weighted than for work layer.

As the first step, we will import all necessary libraries and set up a `matplotlib`'s parameters and notebook configuration: 

In [ ]:
import configparser
from typing import Any

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

np.random.seed(0)

from network_diffusion import (
    MultilayerNetwork,
    MultiSpreading,
    PropagationModel,
)

# set deefault values for matplotlib
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 150 

# read global config
config = configparser.ConfigParser()
config.read("config.ini")
output_dir = config.get("PATHS", "output_dir")

## Network

Then  we  are  able  to  create  a  multilayer  graph  with  two  layers:  ’work’  (`work_layer`)  and ’social network’ (`twtr_layer`).  The first one will be sparser than the second - this operation will simulate a less intensive interaction in the office compared to social life:

In [ ]:
work_layer = nx.karate_club_graph()
nodes_to_remove = [
    (32, 33), 
    (4, 10), 
    (2, 3), 
    (3, 7),
    (23, 27),
    (20, 33),
    (0, 10), 
    (22, 33), 
    (28, 33), 
    (23, 29),
    (2, 7),
    (29, 33),
    (0, 8), 
    (26, 33),
    (1, 7), 
    (0, 1), 
    (15, 33),
    (0, 21), 
    (3, 13), 
    (1, 19),
    (4, 6), 
    (30, 32),
]
work_layer.remove_edges_from(nodes_to_remove)
twtr_layer = nx.karate_club_graph()
network = MultilayerNetwork()
network.load_layers_nx([work_layer, twtr_layer], ["inf_work", "inf_twtr"])

As a result, we will obtain not a multiplex but a multilayer network. That can be proved by visualisation of them with the following code:

In [ ]:
network.describe()

layout = nx.spring_layout(twtr_layer)
fig, ax = plt.subplots(nrows=1, ncols=2)
nx.draw(work_layer, with_labels=True, pos=layout, ax=ax[0])
ax[0].set_title('Work layer')
nx.draw(twtr_layer, with_labels=True, pos=layout, ax=ax[1])
ax[1].set_title('Twtr layer')
plt.show()

## Propagation model

After that, we can create a propagation model. In our story, we distinguished one process, which spreads in two environments: 'work' and 'social network'. The relation between a person (node) and phenomena (gossip) appears in two states: aware (*a*) and unaware (*u*). In this condition, each individual can get to know about the gossip (*u -> a*) with some pseudo-probability of a certain value at work and another in the social, like in the picture below:

<img src="aux/model_real.png">

`Network diffusion` allows modelling phenomena like this. To do it, we must, however, write them in a way understandable to the library. In other words, we will define a two process model (one for each layer), but with weights set up to *1* in two transitions $w_{ua}^{aa} = 1, w_{au}^{aa} = 1$  (first item in their index stands for 'social network' layer and second for 'work' layer):

<img src="aux/model_nd.png">

To simulate faster spreading by the Internet than by real human interactions we will set up $w_{uu}^{au}=0.1$, $w_{uu}^{ua}=0.05$. With that assumption, we are ready to define a propagation model:

In [ ]:
model = PropagationModel()
phenomenas = [["u", "a"], ["u", "a"]]
for l, p in zip(network.get_layer_names(), phenomenas):
    model.add(l, p)

w_background = 0
w_x = 0.05
w_y = 0.1

model.compile(background_weight=w_background)

model.set_transition_fast("inf_work.u", "inf_work.a", (["inf_twtr.u"]), w_x)
model.set_transition_fast("inf_work.u", "inf_work.a", (["inf_twtr.a"]), 1)

model.set_transition_fast("inf_twtr.u", "inf_twtr.a", (["inf_work.u"]), w_y)
model.set_transition_fast("inf_twtr.u", "inf_twtr.a", (["inf_work.a"]), 1)

model.describe()

## Experiment

Now we are ready to set up an experiment. To do it, we have to connect model and network and initialise states of the nodes. We can initialise them randomly or manually by assigning the status of selected nodes.

As starting parameters of simulation, we will set:
* in Social Network layer 30 nodes unaware and 4 aware of gossip
* in Work layer 33 nodes unaware and 1 aware of gossip

in two ways (select either option A or option B)

In [ ]:
# option A - manual selection of nodes; this guarantee that we will obtain reproducible results
def set_node_state(
    experiment: MultiSpreading, layer_name: str, node_name: Any, state: str
) -> None:
    """Allows to set up the initial state of certain node."""
    experiment._network.layers[layer_name].nodes[node_name]["status"] = state


experiment = MultiSpreading(model, network)

experiment.set_initial_states({"inf_twtr": (34, 0), "inf_work": (34, 0)})

set_node_state(experiment, "inf_twtr", 2, "a")
set_node_state(experiment, "inf_twtr", 3, "a")
set_node_state(experiment, "inf_twtr", 4, "a")
set_node_state(experiment, "inf_twtr", 5, "a")

set_node_state(experiment, "inf_work", 2, "a")

In [ ]:
# option B - random selection of nodes; this is a default way to do it in `Network Diffusion`
experiment = MultiSpreading(model, network)
experiment.set_initial_states({"inf_twtr": (30, 4), "inf_work": (33, 1)})

In [ ]:
init_status = experiment._network.get_nodes_states()
logs = experiment.perform_propagation(n_epochs=20)
final_status = experiment._network.get_nodes_states()

print(f"State of nodes before experiment {init_status}")
print(f"State of nodes after  experiment {final_status}")

## Results

After that, we can obtain results:

In [ ]:
logs.report(to_file=True, path=output_dir, visualisation=True)